In [128]:
!pip3 install requests requests_oauthlib foursquare python-dotenv
import datetime
import folium
import foursquare
import json
import numpy as np
import os
import pandas as pd
import requests
import time

from folium.plugins import HeatMap,MarkerCluster

from datetime import date
from dotenv import load_dotenv
from pandas.io.json import json_normalize
from requests_oauthlib import OAuth1Session
from IPython.display import Image,display_jpeg

load_dotenv()

True

In [3]:
# pandas read file
data=pd.read_json("./ロケーション履歴/ロケーション履歴.json","r")

# normalize json to column
d=json_normalize(data['locations'])

d["date"]=d["timestampMs"].apply(lambda x : datetime.datetime.utcfromtimestamp(int(x)/1000))
d["lat"]=d["latitudeE7"].apply(lambda x:x*10**-7)
d["long"]=d["longitudeE7"].apply(lambda x:x*10**-7)



In [124]:

loc_d=d[['lat','long']].values.tolist()

m=folium.Map()
HeatMap(loc_d,radius=15).add_to(m)

m.save('heatmap.html')

In [ ]:
# foursquare
# get oauth token

# 作成したアプリの情報を設定
CLIENT_ID = os.environ.get("CLIENT_ID")
CLIENT_SECRET = os.environ.get("CLIENT_SECRET")
REDIRECT_URI = os.environ.get("REDIRECT_URI")


client = foursquare.Foursquare(client_id=CLIENT_ID, client_secret=CLIENT_SECRET, redirect_uri=REDIRECT_URI)
auth_uri = client.oauth.auth_url()
print(auth_uri)

# 表示されたauth_uriにブラウザからアクセスし、URIの「?code=」の後から「#」の前までの文字列を入力
code=input("INPUT CODE:")
# アクセストークンを取得
access_token = client.oauth.get_token(code)
print(access_token)

# アクセストークンを設定
client.set_access_token(access_token)

# 試しに自分のユーザ情報出力する
# ユーザ情報が表示されたら無事成功
user = client.users()
# print(user)

In [38]:
checkins_ =client.users.all_checkins()
checkins =list(checkins_)

checkin_df=pd.DataFrame.from_dict(checkins)
checkin_df['time']= checkin_df['createdAt'].apply(lambda x: datetime.datetime.fromtimestamp(x))

In [125]:

for venue in checkin_df['venue']:
    try:
        lat,lng=venue['location']['lat'],venue['location']['lng']
        folium.Marker(
            location=[lat,lng],
            popup=""
        ).add_to(m)

    except:
        pass

m.save('heatmap3.html')